In [1]:
import pandas as pd
import numpy as np
import os
import json
from collections import defaultdict

from data_prep_scripts.data_countries import pred_country_data 
from data_prep_scripts.data_manipulation import divide_repetitive_blocks, process_block, df_replaceColVals_vars
from data_prep_scripts.field_derivations import derive_relatives_categ

In [2]:
country_features = pred_country_data()

In [3]:
medData = pd.read_excel('Merged File 2.5.19 De-identified.xlsx')
medData['Country of Origin for Father'].replace(76,'Ireland',inplace=True)
medData = medData[(medData['Breast Cancer?']=='Yes') & (medData['Gender'] == 'Female')]
medData = medData.applymap(lambda s:s.lower() if type(s) == str else s)

In [4]:
predCol = pd.read_excel('Additional BCD Info 3.1.19 De-identified.xlsx',usecols="A,DZ")

In [5]:
medData = medData.merge(predCol,left_on='ID #', right_on='ID #')

In [6]:
ind_repititive_blocks = [
    (47,89,7),(89,117,7),(117,152,7),
    (163,191,7),(197,213,4),(214,249,5),
    (294,348,6),(349,356,7),(357,364,7)
]
ind_not_del = list(range(22,43)) + [274]

In [7]:
familyData1 = medData.iloc[:,47:89]

In [8]:
result = divide_repetitive_blocks(
    familyData1,
    7,
    categ_cols = [0,2,4,5],
    float_cols = [3],
    derive_fields=[(derive_relatives_categ,'relative_categ','categ')],
    null_fields=['Relative:','Age at diagnosis'],
    col_trim_begin=3,
    col_trim_end=0,
    lower_case=True
)

In [9]:
# mixed_cols=[]
# for ii, (a,b) in enumerate(medData.dtypes.iteritems()):
#     if b == object:
#         types = set([type(k) for k in list(medData[a].dropna() ) ])
#         if str in types and len(types)>1: 
#             print(ii,a,b)

In [10]:
# columns that should only contain real numbered values, but contain strings in the raw data.
mixedCols_write_strs = [92,121,126,131,136,141,146,151,152,192,294,303,416,417,418,446,449]
# delcols 
# numbers greater than 30000
mixedCols_write_ints = []

dict_replaceColVals = df_replaceColVals_vars(medData,mixedCols_write_strs,str_vals=True)

In [11]:
# cols_to_del=[]
# for ii, (a,b) in enumerate(medData.dtypes.iteritems()):
#     in_rep_block = False
#     for block in ind_repititive_blocks: 
#         if ii>=block[0] and ii < block[1]: in_rep_block = True
            
#     if not in_rep_block and b == object and ii not in ind_not_del:
#         col_max_count = medData[a].value_counts().iloc[0]
#         if col_max_count < 250: cols_to_del.append(a)
#         elif medData.shape[0]-col_max_count < 250: cols_to_del.append(a)
